In [1]:
from gezi.common import *
sys.path.append('..')
from src.config import *
from src.preprocess import *
from src.eval import *
gezi.init_flags()

In [2]:
# https://www.kaggle.com/code/goldenlock/ai4code-base?scriptVersionId=101148262
# 9025

In [5]:
root = '../working/offline/6/0'
context_model_name = 'deberta-v3-small.flag-context2-aug.n_context-40.cls_loss_rate-0.1.eval.p13-4'
pairwise_model_name = 'all-mpnet-base-v2.flag-pairwise13-2.pooling_mask-attention_mask'

In [6]:
xc = gezi.load(f'{root}/{context_model_name}/valid.pkl')

In [7]:
xp = gezi.load(f'{root}/{pairwise_model_name}/valid.pkl')

In [8]:
gezi.sort_dict_byid_(xc, 'cid')
gezi.sort_dict_byid_(xp, 'cid')

In [9]:
ids = set(xc['id'])

In [10]:
df_gt = pd.read_csv(f'{FLAGS.root}/train_orders.csv')
df_gt = df_gt[df_gt.id.isin(ids)]
df_gt['cell_order'] = df_gt['cell_order'].apply(lambda x: x.split())
df_gt.head()

,id,cell_order
4,0002115f48f982,"[9ec225f0, 18281c6c, e3b6b115, 4a044c54, 365fe..."
11,00062ab8487156,"[dcad687f, a2e1fc80, 7d977ee8, 45a82a59, cbbc3..."
28,000efd285fb982,"[74a30f80, ee2c8e08, 5523374e, ae8f8fe8, 2138e..."
39,0012865b766949,"[f9cb50e9, 25f7db90, d804e819, 6593a545, fc5bb..."
42,001308991e0c5e,"[21147235, 6c01d0d2, 5bd28595, b8fd3a8c, a2501..."


In [11]:
calc_metric(xc, 'reg_pred')

0.9006251692558017

In [12]:
calc_metric(xc, 'pred')

0.8992880214723898

In [13]:
calc_metric(xc, 'cls_pred')

0.896173129530509

In [14]:
calc_metric(xp, 'cls_pred')

0.8931927695789988

In [15]:
x = xc.copy()
x['pred'] = xc['reg_pred'] * 0.5 + xp['cls_pred'] * 0.5
calc_metric(x)

0.9029967267783606

In [16]:
x = xc.copy()
x['pred'] = xc['cls_pred'] * 0.5 + xp['cls_pred'] * 0.5
calc_metric(x)

0.9003450174736564

In [17]:
x = xc.copy()
x['pred'] = xc['reg_pred'] * 0.5 + xp['pred'] * 0.5
calc_metric(x)

0.9023942485028029

In [18]:
def merge(x, y, prob):
  # return y
  # return x
  if prob > 0.9:
    return x * (1 - 0.0001) + y * 0.0001
  elif abs(y - x) < 0.1:
    return x * (1 - 0.0001) + y * 0.0001
  elif abs(y - x) < 0.2:
    return x * 0.95 * prob + y * (1 - 0.95 * prob)
  elif abs(y - x) < 0.3:
    return x * 0.85 * prob + y * (1 - 0.85 * prob)
  elif abs(y - x) < 0.4:
    return x * 0.5 * prob + y * (1 - 0.5 * prob)
  else:
    return y

In [19]:
x['pred'] = [merge(x, y, prob) for x, y, prob in zip(xp['pred'], xc['pred'], xp['max_prob'])]
calc_metric(x)

0.9076464578960967

In [20]:
df_p = pd.DataFrame(xp)

In [21]:
df_c = pd.DataFrame(gezi.batch2list(xc))

In [22]:
df = df_c.merge(df_p[['cid', 'pred', 'cls_pred', 'max_prob', 'max_sim', 'probs', 'sims']], on='cid', suffixes=['_c', '_p'])

In [23]:
df_train = pd.read_feather('../working/train.fea')

In [24]:
df_train = df_train[df_train.id.isin(ids)]

In [25]:
df = df.merge(df_train[['cid', 'ancestor_id', 'n_words', 'source']], on='cid')

In [26]:
gezi.set_fold(df, 5, 'ancestor_id')

In [27]:
df['pred_diff0'] = abs(df['pred_c'] - df['pred_p'])
df['pred_diff1'] = abs(df['reg_pred'] - df['pred_p'])
df['pred_diff2'] = abs(df['cls_pred_c'] - df['pred_p'])
df['pred_diff3'] = abs(df['cls2_pred'] - df['pred_p'])
df['markdown_frac'] = df['n_markdown_cell'] / df['n_cell']
df['span'] = 1 / (df['n_code_cell'] + 1)
top2, top3, top4, top5 = [], [], [], []
top2_prob, top3_prob, top4_prob, top5_prob = [], [], [], []
top2_sim, top3_sim, top4_sim, top5_sim = [], [], [], []
for i in tqdm(range(len(df)), desc='top'):
  # cls_preds = df['cls_pred_ori'].values[i]
  n_code = df['n_code_cell'].values[i]
  probs = df['probs'].values[i]
  sims = df['sims'].values[i]
  idxes = (-probs).argsort()
  if len(idxes) > 1:
    top2.append((idxes[1] + 0.5) / (n_code + 1))
    top2_prob.append(probs[idxes[1]])
    top2_sim.append(sims[idxes[1]])
  else:
    top2.append(-1)
    top2_prob.append(-1)
    top2_sim.append(-1)
  if len(idxes) > 2:
    top3.append((idxes[2] + 0.5) / (n_code + 1))
    top3_prob.append(probs[idxes[2]])
    top3_sim.append(sims[idxes[2]])
  else:
    top3.append(-1)
    top3_prob.append(-1)
    top3_sim.append(-1)
  if len(idxes) > 3:
    top4.append((idxes[3] + 0.5) / (n_code + 1))
    top4_prob.append(probs[idxes[3]])
    top4_sim.append(sims[idxes[3]])
  else:
    top4.append(-1)
    top4_prob.append(-1)
    top4_sim.append(-1)
  if len(idxes) > 4:
    top5.append((idxes[4] + 0.5) / (n_code + 1))
    top5_prob.append(probs[idxes[4]])
    top5_sim.append(sims[idxes[4]])
  else:
    top5.append(-1)
    top5_prob.append(-1)
    top5_sim.append(-1)
ctop_prob, ctop2, ctop3, ctop4, ctop2_prob, ctop3_prob, ctop4_prob = [], [], [], [], [], [], []
for i in tqdm(range(len(df)), desc='ctop'):
  preds = df['cls_pred_ori'].values[i]
  probs = gezi.softmax(preds)
  idxes = (-probs).argsort()
  ctop_prob.append(probs[idxes[0]])
  ctop2.append((idxes[1] + 0.5) / FLAGS.num_classes)
  ctop2_prob.append(probs[idxes[1]])
  ctop3.append((idxes[2] + 0.5) / FLAGS.num_classes)
  ctop3_prob.append(probs[idxes[2]])
  ctop4.append((idxes[3] + 0.5) / FLAGS.num_classes)
  ctop4_prob.append(probs[idxes[3]])
# for i in range(FLAGS.num_classes):
#   df[f'cls_pred{i}'] = df['cls_pred_ori'].apply(lambda x: x[i])
df['top2'] = top2
df['top2_prob'] = top2_prob
df['top2_sim'] = top2_sim
df['top3'] = top3
df['top3_prob'] = top3_prob
df['top3_sim'] = top3_sim
df['top4'] = top4
df['top4_prob'] = top4_prob
df['top4_sim'] = top4_sim
df['top5'] = top5
df['top5_prob'] = top5_prob
df['top5_sim'] = top5_sim
df['ctop_prob'] = ctop_prob
df['ctop2'] = ctop2
df['ctop2_prob'] = ctop2_prob
df['ctop3'] = ctop3
df['ctop3_prob'] = ctop3_prob
df['ctop4'] = ctop4
df['ctop4_prob'] = ctop4_prob
df['pred_diff4'] = abs(df['pred_c'] - df['top2'])
df['pred_diff5'] = abs(df['reg_pred'] - df['top2'])
df['pred_diff6'] = abs(df['cls_pred_c'] - df['top2'])
df['pred_diff7'] = abs(df['cls2_pred'] - df['top2'])
df['pred_diff8'] = abs(df['pred_p'] - df['top2'])
df['pred_diff9'] = abs(df['cls_pred_c'] - df['top3'])
df['pred_diff10'] = abs(df['pred_p'] - df['top3'])
df['rule_pred'] = [merge(x, y, prob) for x, y, prob in tqdm(zip(df.pred_p.values, df.pred_c.values, df.max_prob.values), total=len(df), desc='rule')]

top:   0%|          | 0/424943 [00:00<?, ?it/s]

ctop:   0%|          | 0/424943 [00:00<?, ?it/s]

rule:   0%|          | 0/424943 [00:00<?, ?it/s]

In [28]:
reg_cols =  [
          'n_code_cell',
          'n_markdown_cell',
          'n_cell',
          'cls_pred_c',
          'pred_c',
          'reg_pred',
          'cls2_pred',
          'pred_p',
          'cls_pred_p',
          'rule_pred',
          'pred_diff0',
          'pred_diff1',
          'pred_diff2',
          'pred_diff3',
          'pred_diff4',
          'pred_diff5',
          'pred_diff6',
          'pred_diff7',
          'pred_diff8',
          'pred_diff9',
          'pred_diff10',
          'max_sim',
          'max_prob',
          'markdown_frac',
          'span',
          'top2',
          'top2_prob',
          'top2_sim',
          'top3',
          'top3_prob',
          'top3_sim',
          'top4',
          'top4_prob',
          'top4_sim',
          'top5',
          'top5_prob',
          'top5_sim',
          'ctop_prob', 
          'ctop2', 
          'ctop2_prob', 
        ]

cat_cols = [
          
            ]
label_col = 'rel_rank'
cols = reg_cols + cat_cols

In [29]:
from catboost import CatBoostRegressor, Pool
xgb_params = {'learning_rate': 0.02,
              'reg_lambda': 7.960622217848342e-07, 
              'subsample': 0.7422597612762745,
              'max_depth': 10, 
              'early_stopping_rounds': 500,
              'n_estimators': 10000,
              'cat_features': [],
              'loss_function': 'MAE',
              }

xgb_params2 = {'learning_rate': 0.09827605967564293,'tree_method':'gpu_hist', 'gpu_id':0,
               'early_stopping_rounds': 50,
               'n_estimators': 10000, }

In [30]:
FOLDS = 5

In [31]:
merge_scores = []
scores = []
for fold in tqdm(range(FOLDS)):
  dvalid = df[df.fold==fold]
  dtrain = df[df.fold!=fold]
  X_train = dtrain[cols]
  y_train = dtrain[label_col]
  X_valid = dvalid[cols]
  y_valid = dvalid[label_col]
  model = CatBoostRegressor(**xgb_params)
  model.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
                verbose=500,
                )  
  dvalid['cb_pred'] = model.predict(dvalid[cols])
  x = {'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values}
  x['pred'] = [merge(x, y, prob) for x, y, prob in zip(dvalid.pred_p.values, dvalid.pred_c.values, dvalid.max_prob.values)]
  merge_score = calc_metric(x, 'pred', df_gt)
  ic(abs(x['pred'] - dvalid['rel_rank']).mean())
  score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cb_pred.values})
  ic(fold, merge_score, score, score - merge_score)
  merge_scores.append(merge_score)
  scores.append(score)
  ic(fold, np.asarray(merge_scores).mean(), np.asarray(scores).mean())
  model.save_model(f'../working/cbt/{fold}.cbt')

  0%|          | 0/5 [00:00<?, ?it/s]

0:	learn: 0.2544240	test: 0.2544240	test1: 0.2543404	best: 0.2543404 (0)	total: 407ms	remaining: 1h 7m 51s
500:	learn: 0.0637261	test: 0.0637261	test1: 0.0663100	best: 0.0663100 (500)	total: 1m 35s	remaining: 30m 15s
1000:	learn: 0.0614865	test: 0.0614865	test1: 0.0653747	best: 0.0653746 (999)	total: 3m 12s	remaining: 28m 52s
1500:	learn: 0.0601094	test: 0.0601094	test1: 0.0650864	best: 0.0650842 (1496)	total: 4m 51s	remaining: 27m 29s
2000:	learn: 0.0590524	test: 0.0590524	test1: 0.0650016	best: 0.0649976 (1982)	total: 6m 29s	remaining: 25m 56s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.064997553
bestIteration = 1982

Shrink model to first 1983 iterations.


[07/19/22 18:16:01] 4047567722.py:19 in <module>
                    abs(x['pred'] - dvalid['rel_rank']).mean(): 0.06640286701298789
[07/19/22 18:16:03] 4047567722.py:21 in <module>
                    fold: 0
                    merge_score: 0.9064440002681321
                    score: 0.9092124745641994
                    score - merge_score: 0.0027684742960673026
[07/19/22 18:16:03] 4047567722.py:24 in <module>
                    fold: 0
                    np.asarray(merge_scores).mean(): 0.9064440002681321
                    np.asarray(scores).mean(): 0.9092124745641994


0:	learn: 0.2544544	test: 0.2544544	test1: 0.2543685	best: 0.2543685 (0)	total: 214ms	remaining: 35m 43s
500:	learn: 0.0640191	test: 0.0640191	test1: 0.0649981	best: 0.0649981 (500)	total: 1m 45s	remaining: 33m 11s
1000:	learn: 0.0619186	test: 0.0619186	test1: 0.0640202	best: 0.0640202 (1000)	total: 3m 22s	remaining: 30m 19s
1500:	learn: 0.0603690	test: 0.0603690	test1: 0.0637344	best: 0.0637338 (1486)	total: 4m 46s	remaining: 26m 59s
2000:	learn: 0.0591578	test: 0.0591578	test1: 0.0636126	best: 0.0636111 (1992)	total: 6m 29s	remaining: 25m 57s
2500:	learn: 0.0581340	test: 0.0581340	test1: 0.0635737	best: 0.0635726 (2491)	total: 8m 20s	remaining: 25m
3000:	learn: 0.0574226	test: 0.0574226	test1: 0.0636003	best: 0.0635723 (2554)	total: 9m 55s	remaining: 23m 8s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.06357225843
bestIteration = 2554

Shrink model to first 2555 iterations.


[07/19/22 18:26:18] 4047567722.py:19 in <module>
                    abs(x['pred'] - dvalid['rel_rank']).mean(): 0.06531415207450601
[07/19/22 18:26:20] 4047567722.py:21 in <module>
                    fold: 1
                    merge_score: 0.9073368758755661
                    score: 0.9103981965700243
                    score - merge_score: 0.0030613206944581917
[07/19/22 18:26:20] 4047567722.py:24 in <module>
                    fold: 1
                    np.asarray(merge_scores).mean(): 0.9068904380718491
                    np.asarray(scores).mean(): 0.9098053355671119


...

[07/19/22 18:36:46] 4047567722.py:19 in <module>
                    abs(x['pred'] - dvalid['rel_rank']).mean(): 0.06577511615491055
[07/19/22 18:36:48] 4047567722.py:21 in <module>
                    fold: 2
                    merge_score: 0.9094900364164551
                    score: 0.9128766039201097
                    score - merge_score: 0.003386567503654625
[07/19/22 18:36:48] 4047567722.py:24 in <module>
                    fold: 2
                    np.asarray(merge_scores).mean(): 0.9077569708533844
                    np.asarray(scores).mean(): 0.9108290916847778


0:	learn: 0.2544610	test: 0.2544610	test1: 0.2541931	best: 0.2541931 (0)	total: 449ms	remaining: 1h 14m 47s
500:	learn: 0.0644373	test: 0.0644373	test1: 0.0633387	best: 0.0633387 (500)	total: 2m 21s	remaining: 44m 44s
1000:	learn: 0.0623102	test: 0.0623102	test1: 0.0623976	best: 0.0623976 (1000)	total: 4m 40s	remaining: 41m 58s
1500:	learn: 0.0608762	test: 0.0608762	test1: 0.0620658	best: 0.0620658 (1500)	total: 6m 57s	remaining: 39m 22s
2000:	learn: 0.0596316	test: 0.0596316	test1: 0.0619430	best: 0.0619416 (1983)	total: 8m 45s	remaining: 35m
2500:	learn: 0.0587588	test: 0.0587588	test1: 0.0619083	best: 0.0619060 (2406)	total: 10m 48s	remaining: 32m 24s
3000:	learn: 0.0579399	test: 0.0579399	test1: 0.0619186	best: 0.0619022 (2792)	total: 12m 24s	remaining: 28m 56s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.06190221465
bestIteration = 2792

Shrink model to first 2793 iterations.


[07/19/22 18:50:27] 4047567722.py:19 in <module>
                    abs(x['pred'] - dvalid['rel_rank']).mean(): 0.06394816897503573
[07/19/22 18:50:30] 4047567722.py:21 in <module>
                    fold: 3
                    merge_score: 0.9112763861461699
                    score: 0.9150087659340846
                    score - merge_score: 0.0037323797879147413
[07/19/22 18:50:30] 4047567722.py:24 in <module>
                    fold: 3
                    np.asarray(merge_scores).mean(): 0.9086368246765808
                    np.asarray(scores).mean(): 0.9118740102471046


0:	learn: 0.2543890	test: 0.2543890	test1: 0.2544766	best: 0.2544766 (0)	total: 317ms	remaining: 52m 50s
500:	learn: 0.0637848	test: 0.0637848	test1: 0.0661320	best: 0.0661320 (500)	total: 1m 59s	remaining: 37m 52s
1000:	learn: 0.0615710	test: 0.0615710	test1: 0.0651551	best: 0.0651551 (1000)	total: 3m 36s	remaining: 32m 22s
1500:	learn: 0.0603011	test: 0.0603011	test1: 0.0648632	best: 0.0648631 (1499)	total: 4m 59s	remaining: 28m 18s
2000:	learn: 0.0592920	test: 0.0592920	test1: 0.0647596	best: 0.0647577 (1997)	total: 6m 30s	remaining: 26m 2s
2500:	learn: 0.0584447	test: 0.0584447	test1: 0.0647457	best: 0.0647367 (2207)	total: 8m 14s	remaining: 24m 43s
3000:	learn: 0.0576099	test: 0.0576099	test1: 0.0647292	best: 0.0647284 (2990)	total: 9m 51s	remaining: 22m 59s
3500:	learn: 0.0569395	test: 0.0569395	test1: 0.0647520	best: 0.0647254 (3186)	total: 11m 25s	remaining: 21m 12s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.06472541159
bestIteration = 3186

Shrink mod

[07/19/22 19:02:47] 4047567722.py:19 in <module>
                    abs(x['pred'] - dvalid['rel_rank']).mean(): 0.06635008126049546
[07/19/22 19:02:49] 4047567722.py:21 in <module>
                    fold: 4
                    merge_score: 0.9030462406054968
                    score: 0.9058557359507987
                    score - merge_score: 0.002809495345301949
[07/19/22 19:02:49] 4047567722.py:24 in <module>
                    fold: 4
                    np.asarray(merge_scores).mean(): 0.9075187078623641
                    np.asarray(scores).mean(): 0.9106703553878435


In [32]:
gezi.plot.feature_importance(model, topn=20)

UnboundLocalError: local variable 'feat_importances' referenced before assignment